In [48]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ValidationError
import json
import operator

In [49]:
#load environment variables from a .env file
load_dotenv()

#initiate the language model fir structured output
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [50]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detail feedback for the essay")
    score: int = Field(description="Score out of 10", ge=0, le=10)

In [51]:
structured_model = model.with_structured_output(EvaluationSchema)

In [52]:
essay = """The government of Bangladesh has formulated a national strategy: the National Strategy for Artificial Intelligence – Bangladesh (March 2020) which sets out a vision for AI adoption across sectors such as agriculture, education, healthcare, transport, etc. 
Digital Watch Observatory
+2
The Daily Star
+2
A draft policy: the National Artificial Intelligence Policy 2024 is underway, meant to guide ethical, inclusive, and human‑centred AI development in Bangladesh. 
digibangla.news
+1
Under existing laws: While there is no dedicated AI law yet, there are relevant laws that apply (e.g., the Digital Security Act, 2018 / Cyber Security Act, 2023 and the Bangladesh Telecommunication Act, 2001) which can capture certain AI‑related misuse (such as disinformation, unlawful surveillance) even though they are not AI‑specific. 
The Business Standard
+1
The government has publicly committed to drafting an AI law. For example, the law minister has said a dedicated AI law will be ready by (or around) September of a certain year
"""

In [53]:
# prompt = f"""Evaluate the language quality of follow essay and provide a feedback and assign a score out of 10 \n{essay}"""
# structured_model.invoke(prompt)

In [54]:
class BCSState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    final_evaluation: str
    individual_scores: Annotated[list[int], operator.add] #reduce function
    avg_score: float

In [55]:
def evaluate_language(state: BCSState):
    prompt = f'Evaluate the language quality of follow essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)

    return {'language_feedback': result.feedback, 'individual_scores': [result.score]}

In [56]:
def evaluate_analysis(state: BCSState):
    prompt = f'Evaluate the depth of analysis of follow essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)

    return {'analysis_feedback': result.feedback, 'individual_scores': [result.score]}

In [57]:
def evaluate_clarity(state: BCSState):
    prompt = f'Evaluate the clarity of thought of follow essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    result = structured_model.invoke(prompt)

    return {'clarity_feedback': result.feedback, 'individual_scores': [result.score]}

In [58]:
def final_evaluation(state: BCSState):
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    result = model.invoke(prompt)

    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])


    return {'final_evaluation': result.content, 'avg_score': avg_score}

In [59]:
graph = StateGraph(BCSState)
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('final_evaluation', final_evaluation)


#define the edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_clarity')
graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_clarity', 'final_evaluation')


graph.add_edge('final_evaluation', END)

workflow = graph.compile()


In [60]:
initial_state = {
    'essay': essay
}

final_state = workflow.invoke(initial_state)
print(final_state)

{'essay': 'The government of Bangladesh has formulated a national strategy: the National Strategy for Artificial Intelligence – Bangladesh (March\u202f2020) which sets out a vision for AI adoption across sectors such as agriculture, education, healthcare, transport, etc. \nDigital Watch Observatory\n+2\nThe Daily Star\n+2\nA draft policy: the National Artificial Intelligence Policy 2024 is underway, meant to guide ethical, inclusive, and human‑centred AI development in Bangladesh. \ndigibangla.news\n+1\nUnder existing laws: While there is no dedicated AI law yet, there are relevant laws that apply (e.g., the Digital Security Act, 2018 / Cyber Security Act, 2023 and the Bangladesh Telecommunication Act, 2001) which can capture certain AI‑related misuse (such as disinformation, unlawful surveillance) even though they are not AI‑specific. \nThe Business Standard\n+1\nThe government has publicly committed to drafting an AI law. For example, the law minister has said a dedicated AI law will

In [69]:
print(final_state['avg_score'])

5.666666666666667
